In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [4]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [11]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2",
)

In [12]:
response_message = model.invoke(
    "say hi"
)

print(response_message.content)

Hi! How can I assist you today?


In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

'The main themes in these retrieved docs are:\n\n1. **Task Decomposition**: The process of breaking down complex tasks into smaller, manageable subgoals, which enables efficient handling of large and complicated tasks.\n2. **Planning**: A crucial component of autonomous agent systems, where the agent plans ahead to achieve a task by identifying subgoals and decomposing them into smaller steps.\n3. **Expert Models**: Specialized models that execute on specific tasks and log results, providing expert-level performance in executing complex tasks.\n4. **Autonomous Agent Systems**: The overall system design, which integrates planning, task decomposition, and execution to enable efficient handling of complex tasks.\n5. **Learning and Reflection**: The ability of the agent to learn from mistakes, refine its actions, and improve the quality of final results through self-criticism and self-reflection.\n\nThese themes suggest that autonomous agent systems can be designed to break down complex ta

## Q&A

In [23]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

If the question is out of context, remind the user that they shoudn't ask unrelated questions. In this situation, stress the fact that any abuse of the tool will be reported.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    # here we have {"context": docs, "question": question}
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    # here we have {"context": formatted_docs, "question": question}
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "How to perform task decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

'To perform task decomposition, users can use simple prompting with instructions such as "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?", or utilize task-specific instructions like "Write a story outline." This approach allows the AI assistant to parse user input into various tasks. Additionally, using task-specific instructions helps ensure that the generated output meets specific requirements.'

## Q&A with retrieval

In [24]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

qa_chain.invoke(question)

'There are three approaches to task decomposition: (1) using simple prompting with LLMs, such as "Steps for XYZ.\\n1.", or "What are the subgoals for achieving XYZ?", (2) utilizing task-specific instructions, and (3) incorporating human inputs. These methods enable efficient handling of complex tasks by breaking them down into smaller, manageable subgoals. This process helps an agent plan ahead and execute tasks effectively.'